In [ ]:
import seaborn as sns
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
#plt.style.use('fivethirtyeight');
#plt.rcParams['font.size'] = 14;
#plt.figure(figsize=(12,5));
palette = sns.color_palette('Paired', 10);
import folium
from folium.plugins import HeatMap
from folium.plugins import HeatMapWithTime

 # **Load data:**

In [ ]:
train_1= pd.read_csv('../input/taxi-data/taxi_data/train_gps_points.csv')
train_2= pd.read_csv('../input/taxi-data/taxi_data/train_hire_stats.csv')
test_df= pd.read_csv('../input/taxi-data/taxi_data/test_hire_stats.csv')
zone_df= pd.read_csv('../input/taxi-data/taxi_data/zones.csv')
data_time= train_2['Date']

**correlation of Hour_slot&Hire count**

In [ ]:
## Look Data Distribution
sns.set(font_scale=1.5)
sns.jointplot(train_2['Hour_slot'], train_2['Hire_count'], kind='resid')

In [ ]:
plt.hist2d(train_2['Hour_slot'], train_2['Hire_count'], bins=(50, 50), vmax=1200)
plt.colorbar()
plt.xlabel('Hour slot')
plt.ylabel('Hire Count')

**Turn "Date" to other form**

In [ ]:
time_df= pd.to_datetime(train_2['Date'], format='%Y-%m-%d')
# Time data preparation
train=pd.DataFrame()
train["year"] = time_df.dt.year
train["month"] = time_df.dt.month
train["day"] = time_df.dt.day
train["weekday"] = time_df.dt.weekday
train.head(200)

**correlation of weekday&Hire count**

In [ ]:
plt.hist2d(train['weekday'], train_2['Hire_count'], bins=(50, 50), vmax=1000)
plt.colorbar()
plt.xlabel('WeekDay')
plt.ylabel('Hire Count')

In [ ]:
sns.jointplot(train['weekday'], train_2['Hire_count'], kind='resid',)

**correlation of Month&Hire count**

In [ ]:
sns.jointplot(train['month'], train_2['Hire_count'], kind='resid',)

In [ ]:
plt.hist2d(train['month'], train_2['Hire_count'], bins=(50, 50), vmax=1000)
plt.colorbar()
plt.xlabel('Month')
plt.ylabel('Hire Count')

In [ ]:
''' Working extract time Feature'''
def extract_time_feature(df): 
  time_colum="Datetime"
  df.index= pd.to_datatime(df[time_colum])
  df.index= df.index.tz_convert()
# Data preprocessing
import datetime
# Time data preprocessing
time_df_gps=pd.to_datetime(train_1['Datetime'],utc=True ,infer_datetime_format=True, format='%Y-%m-%d %H-%M-%S +UTC')
# Time data preparation
train=pd.DataFrame()
train["year"] = time_df_gps.dt.year
train["month"] = time_df_gps.dt.month
train["day"] = time_df_gps.dt.day
train["weekday"] = time_df_gps.dt.weekday
train["hour"] = time_df_gps.dt.hour

**correlation of Zone_ID&Hire count**

In [ ]:
sns.jointplot(train_1['Zone_ID'], train_2['Hire_count'], kind='hex')

**correlation of hour &Hire count**

In [ ]:
gps_extract_df=pd.DataFrame()
gps_extract_df['hour']= train['hour']
gps_extract_df['Zone_ID']=train_1['Zone_ID']
gps_extract_df['day']= train['day']
gps_extract_df['month']= train['month']
gps_extract_df['year']=train['year']
gps_extract_df
# generate a table of those culprit rows which are duplicated:
df=gps_extract_df
dups = df.groupby(df.columns.tolist()).size().reset_index().rename(columns={0:'count'})
dups
## Look Data Distribution
sns.jointplot(dups['hour'], dups['count'], kind='hex', ylim=(0,200))

 # **HeatMap of Hire count**

In [ ]:
df=train_1.copy()
df=df.sample(frac=1).reset_index(drop=True)
df=df[:100000]
for col in ["Longitude_X", "Latitude_Y"]:
    MIN = df[col].min()
    MAX = df[col].max()
    print(col, MIN, MAX)
df = df[df["Longitude_X"].between(left = 121.5, right = 121.7 )]
df = df[df["Latitude_Y"].between(left = 24.0, right = 25.2 )] 
center_location = [25.0838005, 121.590033]
m = folium.Map(location=center_location, control_scale=True,zoom_start=13)

In [ ]:
Heat_data = [[row['Latitude_Y'], row['Longitude_X']] for index, row in df.iterrows()]
HeatMap(Heat_data,radius=18).add_to(m)
from folium.plugins import MarkerCluster
marker_cluster = MarkerCluster().add_to(m)
for index, row in zone_df.iterrows():
    information = 'Zone_ID: '+ str(row['Zone_ID'])
    folium.Marker(location=[(row['top']+row['bottom'])/2, (row['left']+row['right'])/2], 
                  popup=folium.Popup(information, max_width=500), max_width='100').add_to(marker_cluster)
m

 # **CircleMap of Hire count**

In [ ]:
df= pd.read_csv('../input/taxi-data/taxi_data/train_hire_stats.csv')
df['Zone_ID']=df['Zone_ID']-1
df=df.groupby(by=['Zone_ID']).sum()
df['Latitude_Y']=(zone_df['top']+zone_df['bottom'])/2
df['Longitude_X']=(zone_df['left']+zone_df['right'])/2
#df=df[:24]
df
m2 = folium.Map(location=center_location, control_scale=True,zoom_start=13)
#for i in range(0,len(df)):
for i in range(len(df)):    
    folium.Circle(
        location=[df.iloc[i]['Latitude_Y'], df.iloc[i]['Longitude_X']],
        tooltip = "<h5 style='text-align:center;font-weight: bold'>"+str(i+1)+"</h5>",
        radius=(int((np.log(df.iloc[i,-3]+1.00001)))+0.2)*100,
        color='#ff6600',
        fill_color='#ff8533',
        fill=True).add_to(m2)

m2

 # **Trend of Hire count**

In [ ]:
df= pd.read_csv('../input/taxi-data/taxi_data/train_hire_stats.csv')
df=df.groupby(by=['Date']).sum()
df=df.drop(["Zone_ID"], axis=1)
df=df.drop(["Hour_slot"], axis=1)

# Plot
f = plt.figure(figsize=(15,10))
ax = f.add_subplot(111)

date = np.arange(0,len(df))
marker_style = dict(linewidth=2, linestyle='-', marker='o',markersize=5)
plt.plot(date,df,"-.",color="red",**marker_style)
ax.tick_params(which='both', width=1,labelsize=12)
ax.tick_params(which='major', length=6)
ax.tick_params(which='minor', length=3, color='0.8')
#plt.xticks(list(np.arange(0,len(total),int(len(total)/5))),total[:-1:int(len(total)/5)]+[total[-1]])
# Grid
plt.grid(lw = 1, ls = '-', c = "0.85", which = 'major')
plt.grid(lw = 1, ls = '-', c = "0.95", which = 'minor')

# Axis Lable
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%m/%d'))
plt.xlabel("Day",fontsize =18)
plt.ylabel("hire_count",fontsize =18)

# plt.yscale("log")
plt.tick_params(labelsize = 13) 
#plt.savefig(out+"daily confirmed cases global.png")
plt.show()

# **NORMALIZE DATA TURN DATA INTO FORMAT MODEL UNDERSTAND WINDOWWINDOW_STRIDE**

**Time Processing turn into Sine and Cosine Frequency**

In [ ]:
'''Removing the Repeat in Time '''
time_df=pd.DataFrame()
time_df['date']= train_2['Date']
time_df['zone_id']=train_2['Zone_ID']
# print('Unique values of ZoneID:', time_df.zone_id.unique())
time_df_rep = time_df.groupby(time_df.columns.tolist()).size().reset_index().rename(columns={0:'count'})
print(time_df_rep)

**This Time Cosine Fequ for 1 YEAR DATA WITHOUT ZONE_ID**

In [ ]:
'''Removing the Repeat in Time '''
time_df=pd.DataFrame()
time_df['date']= train_2['Date']
time_df['zone_id']=train_2['Zone_ID']
# print('Unique values of ZoneID:', time_df.zone_id.unique())
time_df_rep = time_df.groupby(time_df.columns.tolist()).size().reset_index().rename(columns={0:'count'})
print(time_df_rep)

''' Get Training Data time '''
holiday=['2016-02-06','2016-02-07','2016-02-08','2016-02-09','2016-02-10','2016-02-11','2016-02-12','2016-02-13','2016-02-14','2016-02-27','2016-02-28','2016-02-29',
        '2016-04-02','2016-04-03','2016-04-04','2016-04-05','2016-06-09','2016-06-10','2016-06-11','2016-06-12',
        '2016-09-15','2016-09-16','2016-09-17','2016-09-18',
         '2016-10-09', '2016-10-10', '2016-10-11','2016-12-31','2017-01-01','2017-01-02',
         '2017-01-27','2017-01-28','2017-01-29','2017-01-30','2017-01-31',
         '2017-02-01','2017-02-25','2017-02-26','2017-02-27','2017-02-28'
        ]
time_df= pd.to_datetime(time_df_rep['date'], format='%Y-%m-%d')
train_time_df=pd.DataFrame()
train_time_df["year"] = time_df.dt.year
train_time_df["month"] = time_df.dt.month
train_time_df["day"] = time_df.dt.day
train_time_df["weekday"] = time_df.dt.weekday
train_time_df["workday"] = time_df.dt.weekday
for i in train_time_df.index: 
     if train_time_df["weekday"][i]==5 or  train_time_df["weekday"][i]==6 or str(time_df.iloc[i]).split()[0] in holiday :
        train_time_df["workday"][i]=0
     else:
        train_time_df["workday"][i]=1
#train["hour"]=train_2['Hour_slot']

'''Convert Date Time into the Frequency Sine and COSINE '''

freq_df=pd.DataFrame()
freq_df['day_sin'] = np.sin(train_time_df.day*(2.*np.pi/30*25))
freq_df['day_cos'] = np.cos(train_time_df.day*(2.*np.pi/30*25))
# Hour
freq_df['hour_sin'] = np.sin(train_time_df.day*(2.*np.pi/24*25))
freq_df['hour_cos'] = np.cos(train_time_df.day*(2.*np.pi/24*25))

plt.plot(np.array(freq_df['hour_sin'][0:720]), scaley=True, scalex=True)
plt.plot(np.array(freq_df['hour_cos'][0:720]))
plt.xlabel('Time [h]')
plt.title('Time of day signal')

In [ ]:
time_df=pd.DataFrame()
time_df['date']= train_2['Date']

time_df= pd.to_datetime(time_df['date'], format='%Y-%m-%d')
train_time_df=pd.DataFrame()
holiday=['2016-02-06','2016-02-07','2016-02-08','2016-02-09','2016-02-10','2016-02-11','2016-02-12','2016-02-13','2016-02-14','2016-02-27','2016-02-28','2016-02-29',
        '2016-04-02','2016-04-03','2016-04-04','2016-04-05','2016-06-09','2016-06-10','2016-06-11','2016-06-12',
        '2016-09-15','2016-09-16','2016-09-17','2016-09-18',
         '2016-10-09', '2016-10-10', '2016-10-11','2016-12-31','2017-01-01','2017-01-02',
         '2017-01-27','2017-01-28','2017-01-29','2017-01-30','2017-01-31',
         '2017-02-01','2017-02-25','2017-02-26','2017-02-27','2017-02-28'
        ]
train_time_df["weekday"] = time_df.dt.weekday
train_time_df["workday"] = 0
for i in train_time_df.index: 
     if train_time_df["weekday"][i]==5 or  train_time_df["weekday"][i]==6 or str(time_df.iloc[i]).split()[0] in holiday :
        train_time_df["workday"][i]=0
     else:
        train_time_df["workday"][i]=1
train_time_df["month"] = time_df.dt.month
train_time_df["zone_id"] = train_2['Zone_ID']
df_month_zone=pd.DataFrame()
df_month_zone['month']=train_time_df["month"] 
df_month_zone['zone']=train_time_df["zone_id"] 

In [ ]:
df_month_zone_rep = df_month_zone.groupby(df_month_zone.columns.tolist()).size().reset_index().rename(columns={0:'count'})
df_month_zone_rep

**Create Cosine Value for 1 DAY Hour value**

In [ ]:
df=[int(i) for i in np.linspace(0, 23, 24)]
df=pd.DataFrame(df)
df
seconds_in_day =24
df['sin_time'] = np.sin(2*np.pi*df[0]/seconds_in_day)
df['cos_time'] = np.cos(2*np.pi*df[0]/seconds_in_day)

plt.plot(np.array(df['sin_time']), scaley=True, scalex=True)
plt.plot(np.array(df['cos_time']))
plt.xlabel('Time [h]')
plt.title('Time of day signal')

In [ ]:
hour_df_freq=pd.DataFrame()
hour_df_freq['hour_sine'] = pd.concat([df['sin_time']]*9150, ignore_index=True)
hour_df_freq['hour_cosine'] = pd.concat([df['cos_time']]*9150, ignore_index=True)
hour_df_freq

In [ ]:
day_fre_df=pd.DataFrame()
day_fre_df['day_sine'] = pd.concat([freq_df['hour_sin']]*24, ignore_index=True)
day_fre_df['day_cosine'] = pd.concat([freq_df['hour_cos']]*24, ignore_index=True)
day_fre_df

In [ ]:
train_2["workday"]=train_time_df["workday"] 

In [ ]:

train_df=train_2.drop(['Hour_slot','Date'], axis=1)
train_df= pd.concat([train_df, day_fre_df], axis=1)
train_df=pd.concat([train_df, hour_df_freq], axis=1)


In [ ]:
train_df=train_df[['day_cosine', 'day_sine','hour_sine', 'hour_cosine','Zone_ID','workday','Hire_count']]
train_df.to_csv('processing_train_df.csv')

In [ ]:
train_df

# **Normalize Splitting Data INTO Training and Testing DATA**
We will using 80% data for training data, 20% for validation

1. Ensure that chopping data into windows of consecutive sample is still possible
2.Ensure that Training/Validation data will not shuffle because due to the time sereis data --> have time and Position is matter Most

In [ ]:
from sklearn.preprocessing import minmax_scale
import pandas as pd
##Normalize Data 
train_df[['Hire_count', 'Zone_ID']]= minmax_scale(train_df[['Hire_count', 'Zone_ID']],feature_range=(0, 1))
#label_df['Hire_count']=train_df['Hire_count']
#train_df=train_df.drop(['Hire_count'], axis=1)
#label_df

In [ ]:
train_df

In [ ]:
n=len(train_df)
training_df= train_df[0:int(n*0.8)]
val_df= train_df[int(n*0.8):]
print(len(training_df))
print(len(val_df))

In [ ]:
training_df

# **Normalize and Preprocessing for Test data**

In [ ]:
test_df

In [ ]:
'''Removing the Repeat in Time '''
test_time_df=pd.DataFrame()
test_time_df['date']= test_df['Date']
test_time_df['zone_id']=test_df['Zone_ID']
# print('Unique values of ZoneID:', time_df.zone_id.unique())
test_time_df_rep = test_time_df.groupby(test_time_df.columns.tolist()).size().reset_index().rename(columns={0:'count'})
print(test_time_df_rep)

In [ ]:
# '''Removing the Repeat in Time '''
# time_df=pd.DataFrame()
# time_df['date']= train_2['Date']
# time_df['zone_id']=train_2['Zone_ID']
# # print('Unique values of ZoneID:', time_df.zone_id.unique())
# time_df_rep = time_df.groupby(time_df.columns.tolist()).size().reset_index().rename(columns={0:'count'})
# print(time_df_rep)

''' Get Training Data time '''
time_df_for= pd.to_datetime(test_time_df_rep['date'], format='%Y-%m-%d')
time_df=pd.DataFrame()
time_df["year"] = time_df_for.dt.year
time_df["month"] = time_df_for.dt.month
time_df["day"] = time_df_for.dt.day
time_df["weekday"] = time_df_for.dt.weekday
#train["hour"]=train_2['Hour_slot']

'''Convert Date Time into the Frequency Sine and COSINE '''

freq_df=pd.DataFrame()
freq_df['day_sin'] = np.sin(time_df.day*(2.*np.pi/30*25))
freq_df['day_cos'] = np.cos(time_df.day*(2.*np.pi/30*25))
# Hour
freq_df['hour_sin'] = np.sin(time_df.day*(2.*np.pi/24*25))
freq_df['hour_cos'] = np.cos(time_df.day*(2.*np.pi/24*25))

plt.plot(np.array(freq_df['hour_sin'][0:720]), scaley=True, scalex=True)
plt.plot(np.array(freq_df['hour_cos'][0:720]))
plt.xlabel('Time [h]')
plt.title('Time of day signal')

# **Building DATA Windowing for Training Model**

**Main feature of Windows input**

1.The width (number of time steps) the input -label Windows

2.The time offset between them

3.Which features are used as inputs, labels, or both

4.Single-Output and Mult-output predictions

5.Single-time-Step and Multi-time-Step predictions


In [ ]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df=train_df, val_df=val_df, 
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df


    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

In [ ]:
w1 = WindowGenerator(input_width=6, label_width=1, shift=1,train_df=training_df, val_df=val_df, 
                     label_columns='Hire_count')
w1

**1.SPLIT_WINDOW_METHOD Convert All the data into the window input and Label**

**2.PLOT function Visualize Data**

1_Spliting Data

Solving model for different step input and Output Data

In [ ]:
## Spliting the Window as we feed the input 
def split_window(self, features): 
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[self.label_columns]]],axis=-1)
    # labels = tf.stack(
    #     [labels[:, :, self.column_indices[name]] for name in self.label_columns],
    #     axis=-1)
  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

import tensorflow as tf
'''IMPORTANT OF IMPROVING YOUR CODE '''
## Learning cool way to Extend your Objects 
WindowGenerator.split_window= split_window

# Stack three slices, the length of the total window:
example_window = tf.stack([np.array(train_df[:w1.total_window_size]),
                           np.array(train_df[100:100+w1.total_window_size]),
                           np.array(train_df[200:200+w1.total_window_size])])

## Example input and Output. 
example_inputs, example_labels= w1.split_window(example_window)
print('All shapes are: (batch, time, features)')
print(f'Window shape: {example_window.shape}')
print(f'Inputs shape: {example_inputs.shape}')
print(f'labels shape: {example_labels.shape}')

2.PLOT_Function

In [ ]:
def plot(self, model=None, plot_col='Hire_count', max_subplots=3):
  inputs, labels = self.example
  print(inputs.shape)
  plt.figure(figsize=(12, 8))
  plot_col_index = self.column_indices[plot_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(max_n, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]')
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
    else:
      label_col_index = plot_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_col_index],
                edgecolors='k', label='Labels', c='#2ca02c', s=64)
    
    if model is not None:
      predictions = model.predict(inputs)
      #print(Predictions.shape)
      plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                  marker='X', edgecolors='k', label='Predictions',
                  c='#ff7f0e', s=64)

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot = plot

In [ ]:
## this line code set attribute example for Windowgenerator 
w1.example = example_inputs, example_labels

In [ ]:
def plot(self, model=None, plot_col='Hire_count', max_suplots=2):
  inputs, labels= self.example
  #print(inputs)

  plt.figure(figsize=(12, 8))
  plot_col_index= self.column_indices[plot_col]
  print(plot_col_index)
  max_n= min(max_suplots, len(inputs))
  
  for n in range(max_n):
    plt.subplot(max_n, 1, n+1)
    plt.ylabel(f'{plot_col}: [Norm]')

    plt.plot(self.input_indices, inputs[n, :, plot_col_index], 
             label='Inputs', marker=".", zorder=-10)
    
    if self.label_columns: 
      #label_col_index = self.label_columns_indices.get(plot_col, None)
      label_col_index = self.label_columns_indices.get(plot_col)
      print(label_col_index)
    else: 
      label_col_index= plot_col_index
      
    if label_col_index is None: 
      continue
    
    plt.scatter(self.label_indices, labels[n, :, label_col_index],
                edgecolors='k', label='Labels', c='#2ca02c', s=64)
    #if model is not None:
    predictions_, predictions= model.predict(inputs)## Consider adding predict??
    print(Predictions)
    print(loss)
    plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                marker='X', edgecolors='k', label='Predictions',
                c='#ff7f0e', s=64)

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

  plt.xlabel('Time [h]')
  
WindowGenerator.plot=plot

In [ ]:
w1.plot()

# **Create TF.data.Dataset -- Keras Processing Time Series Data**
**Purpose TURN Dataframe to tf.data.Dataset (input_window, label_window)**

**tf.data.Dataset you can easily iterate over the Data --> Dataset.element_spec property tell you the structure, dtypes and shape to the dataset elements**


In [ ]:
'''Convert data to tensor with certain batch_size'''
def make_dataset(self, data): 
  ## Convert data to Numpy array
  data= np.array(data, dtype= np.float32)

  ds= tf.keras.preprocessing.timeseries_dataset_from_array(
      data=data, 
      targets=None, 
      sequence_length= self.total_window_size, 
      sequence_stride=1, 
      shuffle= True, 
      batch_size=32, )
  ds= ds.map(self.split_window)
  return ds

WindowGenerator.make_dataset= make_dataset
'''The WindowGenerator adding property to access training - val datset'''
## Traindata
@property
def train(self): 
  return self.make_dataset(self.train_df)
#Valdata
@property
def val(self): 
  return self.make_dataset(self.val_df)
@property
def example(self): 
  """Get and Cache an example batch of 'Input', 'labels' for plotting"""
  result= getattr(self, '_example', None)
  if result is None: 
    #No example batch was found, so get from the train dataset
    result= next(iter(self.train))
    #cache it for the next time
    self._example= result
  return result

WindowGenerator.train=train
WindowGenerator.val=val
WindowGenerator.example=example

In [ ]:
w1.plot()

In [ ]:
#1 Inspecting data 
print(w1.train.element_spec)
#2 Iterate over dataset and Yield concrete batches
for example_inputs, example_labels in w1.train.take(1): 
  print(f'Inputs shape (batch, time, features): {example_inputs.shape}')
  print(f'Labels shape(batch, time, features): {example_labels.shape}')

# **Model window Generator will Base On Two Approach**
**1.Single Steps model**

**2.Multi-steps input and outputs models**

**3.Multiple steps Mode**

![](http://)

In [ ]:
def compile_and_fit(model, window, patience=20):
  #early_stopping= tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
  #                                                 patience= patience, mode='min')
  model.compile(loss=tf.losses.MeanSquaredError(), 
                optimizer=tf.optimizers.Adam(), 
                metrics=[tf.metrics.MeanAbsoluteError()])
  history= model.fit_generator(window.train, epochs=EPOCHS, 
                     validation_data=window.val, callbacks=[early_stopping]
                     )
  return history

In [ ]:
import tensorflow as tf
Dense = tf.keras.Sequential([
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=1)
])

multiple_steps_window = WindowGenerator(
    input_width=24, label_width=24, shift=1,
    train_df=training_df, val_df=val_df, 
                     label_columns='Hire_count')

print('Input shape', multiple_steps_window.example[0].shape)
print('Output shape', Dense(multiple_steps_window.example[0]).shape)


def compile_and_fit(model, window, patience=20):
  #early_stopping= tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
#                                                   patience= patience, mode='min')
  model.compile(loss=tf.losses.MeanSquaredError(), 
                optimizer=tf.optimizers.Adam(), 
                metrics=[tf.metrics.MeanAbsoluteError()])
 # history= model.fit_generator(window.train, epochs=EPOCHS, 
  #                   validation_data=window.val, callbacks=[early_stopping])
  history= model.fit_generator(window.train, epochs=EPOCHS, 
                     validation_data=window.val)
                     
  return history, model
EPOCHS=50
val_performance = {}
history, model= compile_and_fit(Dense, multiple_steps_window)
val_performance['Dense_multiple_steps'] = Dense.evaluate(multiple_steps_window.val)

In [ ]:
Dense.save_weights('Dense_multiple_steps_ouputs.h5')
multiple_steps_window.plot(model= Dense)

In [ ]:
## We need to Connect all Three Multiple timestep together FLATTEN layer
Dense_single = tf.keras.Sequential([
    # Connect all multiple steps flatten()                     
    tf.keras.layers.Flatten(),               
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=1),
    ## there also need to reshape the output layers 
    tf.keras.layers.Reshape([1, -1])
])

## We Using 6hours to Predict 1 hours Outputs 
sing_step_out_window = WindowGenerator(
    input_width=4, label_width=1, shift=1,
    train_df=training_df, val_df=val_df, 
                     label_columns='Hire_count')

print('Input shape', sing_step_out_window.example[0].shape)
print('Output shape', Dense(sing_step_out_window.example[0]).shape)

EPOCHS=50
#val_performance = {}
history= compile_and_fit(Dense_single, sing_step_out_window)
val_performance['Dense_Single_step'] = Dense_single.evaluate(sing_step_out_window.val)
Dense_single.save_weights('Dense_Single_steps_output.h5')

In [ ]:
## We Using 6hours to Predict 1 hours Outputs 
sing_step_out_window = WindowGenerator(
    input_width=4, label_width=1, shift=1,
    train_df=training_df, val_df=val_df, 
                     label_columns='Hire_count')
sing_step_out_window.plot(model=Dense)

In [ ]:
'''Create Window Generator for Conv1D model'''
## 3 input steps and 1 output steps
conv_width=4
conv_window=WindowGenerator(input_width=conv_width, label_width=1, shift=1,
    train_df=training_df, val_df=val_df, 
                     label_columns='Hire_count')
print(conv_window)
print("Conv model on conv Window")
print("input_shape: ", conv_window.example[0].shape)

Conv1D_model= tf.keras.Sequential([
                                   tf.keras.layers.Conv1D(filters=64, kernel_size=(conv_width,), activation='relu' ),
                                   tf.keras.layers.Dense(units=64, activation='relu'), 
                                   tf.keras.layers.Dense(units=32, activation='relu'), 
                                   tf.keras.layers.Dense(units=1), 
])
print("Output_shape", Conv1D_model(conv_window.example[0]).shape)

EPOCHS=50
#val_performance = {}
history= compile_and_fit(Conv1D_model, conv_window)
val_performance['Conv1_Single_step'] = Conv1D_model.evaluate(conv_window.val)
Conv1D_model.save_weights('Conv1D_single_steps_output_2hidden_Dense.h5')

In [ ]:
conv_window.plot(Conv1D_model)

In [ ]:
conv_width=4
LABEL_WIDTH = 24
INPUT_WIDTH = LABEL_WIDTH + (conv_width - 1)

wide_conv_window=WindowGenerator(input_width=INPUT_WIDTH, label_width=LABEL_WIDTH, shift=1,
    train_df=training_df, val_df=val_df, 
                     label_columns='Hire_count')
print(wide_conv_window)
print("Conv model on conv Window")
print("input_shape: ", wide_conv_window.example[0].shape)
print("Output_shape", Conv1D_model(wide_conv_window.example[0]).shape)

EPOCHS=50
val_performance = {}
history= compile_and_fit(Conv1D_model, wide_conv_window)
val_performance['Conv1D_Multiple_steps'] = Conv1D_model.evaluate(wide_conv_window.val)
Conv1D_model.save_weights('Conv1D_multiple_steps_output_2hidden.h5')

In [ ]:
''' LSTM model will training on the same 
1 Conv Window // 4 hours Input -- predict 1 hour Output 
2 wide Conv Window // 4 hours input -- 4 Hours outputs
'''
'''SECTION 1 Train on Multiple inputs steps 1 outputs steps'''
# [[Model can improve by adding more layers and Neuron units]]
# lstm_model= tf.keras.Sequential([
#                                  tf.keras.layers.LSTM(64, return_sequences=True),
#                                  tf.keras.layers.Dense(units=1)
# ])
# print('Input shape:', conv_window.example[0].shape)
# print('Output shape:', lstm_model(conv_window.example[0]).shape)
# ## this Mod
# EPOCHS=50
# val_performance = {}
# history= compile_and_fit(lstm_model, conv_window)
# val_performance['LSTM'] = lstm_model.evaluate(conv_window.val)
# lstm_model.save_weights('LSTM_Single_steps_output.h5')

'''SECTION 1 Train on Multiple inputs steps and Multiple outputs steps'''
num_features=6
lstm_model= tf.keras.Sequential([
                          tf.keras.layers.LSTM(32, return_sequences=True),
                           tf.keras.layers.LSTM(32, return_sequences=True),
                          # Shape => [batch, time, features]
                          tf.keras.layers.Dense(units=num_features)
])
print('Input shape:', wide_conv_window.example[0].shape)
print('Output shape:', lstm_model(wide_conv_window.example[0]).shape)
## this Mod
# EPOCHS=50
# val_performance = {}
# history= compile_and_fit(lstm_model, wide_conv_window)
# val_performance['LSTM'] = lstm_model.evaluate(wide_conv_window.val)
# lstm_model.save_weights('LSTM_Multiple_steps_output.h5')

In [ ]:
## Define multiple steps OUTPUT 
OUT_STEPS= 24
hour_input= 24
num_features=6
multi_window= WindowGenerator(input_width =hour_input, label_width= OUT_STEPS, train_df=training_df,
                              val_df= val_df, shift= OUT_STEPS)
multi_window.plot()

### Model Architecture will Train on Dense -- CNN -- LSTM model 
## Dense Model 
multi_dense_model= tf.keras.Sequential([
                                  #Take the last time steps 
                                  #shape [Batch_size,  time, features] => [batch, 1, features]
                                  tf.keras.layers.Lambda(lambda x: x[:, -1:, :]),
                                  #shape => > [batch, 1, dense_units]
                                  tf.keras.layers.Dense(512, activation='relu'), 
                                  tf.keras.layers.Dense(214, activation='relu'), 
                                  #Shape ==> [batch, 1, out_steps*features]
                                  tf.keras.layers.Dense(OUT_STEPS*num_features, 
                                                        kernel_initializer= tf.initializers.zeros()), 
                                  #shape=> [batch, out_steps, features]
                                  tf.keras.layers.Reshape([OUT_STEPS, num_features])
                                  
])
history= compile_and_fit(multi_dense_model, multi_window)
EPOCHS=50
val_performance_mutliple_steps = {}
val_performance_mutliple_steps['Multi_Dense'] = multi_dense_model.evaluate(multi_window.val)
multi_dense_model.save('Multi_Dense_steps_output.h5')

In [ ]:
## Model 1 layer CONV1D and 1 Layer Hidden 

CONV_WIDTH=3
multi_conv_model= tf.keras.Sequential([
                                       #shape [batch, time, conv_Units] => [batch, CONV_WIDTH, features]
                                       tf.keras.layers.Lambda(lambda x:  x[:, -CONV_WIDTH:, :]), 
                                       #Shape => [batch, 1, conv_units]
                                       tf.keras.layers.Conv1D(256, activation='relu', kernel_size=(CONV_WIDTH)), 
                                       #Shape => [batch, 1, out_steps*features ]
                                       tf.keras.layers.Dense(units=256,  activation='relu'),
                                       tf.keras.layers.Dense(OUT_STEPS*num_features, kernel_initializer=tf.initializers.zeros()), 
                                       #shape => [batch, out_step, features]
                                       tf.keras.layers.Reshape([OUT_STEPS, num_features])    
])

EPOCHS=50
history= compile_and_fit(multi_dense_model, multi_window)
val_performance_mutliple_steps = {}
val_performance_mutliple_steps['Multi_Conv_Output'] = multi_dense_model.evaluate(multi_window.val)
multi_dense_model.save('Multi_Conv_steps_output.h5')


In [ ]:
multi_lstm_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, lstm_units]
    # Adding more `lstm_units` just overfits more quickly.
    tf.keras.layers.LSTM(64, return_sequences=False),
    tf.keras.layers.Dense(units=256, activation='relu'), 
    # Shape => [batch, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*num_features,
                          kernel_initializer=tf.initializers.zeros()),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, num_features])
])

history = compile_and_fit(multi_lstm_model, multi_window)
# IPython.display.clear_output()

multi_val_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.val)
multi_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.test, verbose=0)
multi_window.plot(multi_lstm_model)

In [ ]:
num_features=6
class FeedBack(tf.keras.Model):
  def __init__(self, units, out_steps):
    super().__init__()
    self.out_steps = out_steps
    self.units = units
    self.lstm_cell = tf.keras.layers.LSTMCell(units)
    # Also wrap the LSTMCell in an RNN to simplify the `warmup` method.
    self.lstm_rnn = tf.keras.layers.RNN(self.lstm_cell, return_state=True)
    self.dense = tf.keras.layers.Dense(num_features)

In [ ]:
feedback_model= FeedBack(units=32, out_steps=OUT_STEPS)
def warmup(self, inputs): 
  #inputs.shape => (batch, time, features)
  #x.shape => (batch, lstm_units)
  x, *state= self.lstm_rnn(inputs)

  #predictions.shape => (batch, features)
  prediction= self.dense(x)
  return prediction, state
FeedBack.warmup= warmup

prediction, state= feedback_model.warmup(multi_window.example[0])
print(prediction.shape)

def call(self, inputs, training=None):
  # Use a TensorArray to capture dynamically unrolled outputs.
  predictions = []
  # Initialize the lstm state
  prediction, state = self.warmup(inputs)
  
  # Insert the first prediction
  predictions.append(prediction)

  # Run the rest of the prediction steps
  for n in range(1, self.out_steps):
    # Use the last prediction as input.
    x = prediction
    # Execute one lstm step.
    x, state = self.lstm_cell(x, states=state,
                              training=training)
    # Convert the lstm output to a prediction.
    prediction = self.dense(x)
    # Add the prediction to the output
    predictions.append(prediction)

  # predictions.shape => (time, batch, features)
  predictions = tf.stack(predictions)
  # predictions.shape => (batch, time, features)
  predictions = tf.transpose(predictions, [1, 0, 2])
  return predictions

FeedBack.call = call

In [ ]:
for n in range (1,self.out_step):
    x= prediction
    # Use the last prediction 
    x , state = self.lstm_cell(x,states=state,
                               training=training)
    predictions = tf.stack(predictions)
    predictions = tf.stack(predictions)
    # Convert the lstm output to a prediction.
    predictions.append(prediction)
    # Add the prediction to the output
    predictions.append(prediction)
# prediction.shape =>(time, batch, features)
predictions = tf.transpose(prediction, [1, 0, 2])
return predictions

FeedBack.call = call
 